In [1]:
#coding=utf-8
import numpy as np
import pandas as pd

## 1.获取数据

In [2]:
## 用户列表
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_earlier6month.csv').decode('utf-8')
data_early = pd.read_csv(path, names=['shopid','list_user'])
data_early.head()

,shopid,list_user
0,1427469939,15026|1703212|5176157|5787401|6222925|6241420|...
1,1427495767,591220876|1101789123|1486286273|1544889055|172...
2,1427538754,1410509|2285814|2783100|14590476|15602920|2151...
3,1427550629,2129564|35210648|253893716|733200958|745665947...
4,1427582876,1096213|1410509|1478861|2285814|4469109|679745...


In [3]:
## 销量
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_later6month.csv').decode('utf-8')
data_later = pd.read_csv(path)
data_later.head()

,店铺id,time,销量
0,1427469939,2015-11,484
1,1427469939,2015-12,294
2,1427469939,2016-01,218
3,1427469939,2016-02,112
4,1427469939,2016-03,162


In [4]:
# 生成list_shop
list_shop = list(set(data_early['shopid'].values))
# 生成dict{shopid: user_sacle}
dict_shop_scale = {}
for shop in list_shop:
    scale = len(set(data_early[data_early['shopid']==shop]['list_user'].values[0].split('|')))
    dict_shop_scale[shop] = scale

In [5]:
# 每月市场大小
list_month = sorted(list(set(data_later['time'].values)))
temp = data_later.groupby('time').sum()
dict_month_scale = temp.to_dict()
print dict_month_scale['销量']

{'2016-07': 86481, '2016-06': 78189, '2016-05': 76441, '2016-04': 65260, '2016-03': 61718, '2016-02': 22925, '2016-01': 37808, '2015-11': 25513, '2016-10': 53571, '2015-12': 32135, '2016-09': 59773, '2016-08': 68247}


## 2.计算竞争强度

In [6]:
# 生成了两两竞争强度（用户重合度），并保存
# path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
# f = open(path, 'w')
# f.write('shop1,shop2,scale1,scale2,coincide\r\n')
# for shop1 in list_shop:
#     for shop2 in list_shop:
#         if shop1 == shop2: continue
#         user1 = set(data_early[data_early['shopid']==shop1]['list_user'].values[0].split('|'))
#         user2 = set(data_early[data_early['shopid']==shop2]['list_user'].values[0].split('|'))
#         f.write(str(shop1) + ',' + str(shop2) + ','\
#                 + str(len(user1)) + ',' + str(len(user2)) + ','\
#                 + str(len(user1 & user2)) + '\r\n')
# f.close()
# # 读取两两竞争强度
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
data_coincide = pd.read_csv(path)

In [7]:
# 计算相对竞争强度
data_coincide['competitor1'] = data_coincide['coincide'] / data_coincide['scale1'] ##2对1的竞争压力
data_coincide['competitor2'] = data_coincide['coincide'] / data_coincide['scale2'] ##1对2的竞争压力
data_coincide = data_coincide.set_index(['shop1', 'shop2'])

In [8]:
data_coincide.head()

scale1  scale2  coincide  competitor1  competitor2
shop1      shop2                                                         
1459224577 1542234114    1319       8         0     0.000000     0.000000
           1458774019    1319     247         0     0.000000     0.000000
           1585268741    1319       7         0     0.000000     0.000000
           1550602246    1319     147         0     0.000000     0.000000
           1441814538    1319     155         1     0.000758     0.006452

In [ ]:
## 绘制分布图
import matplotlib.pyplot as plt
temp = list(data_coincide[data_coincide['competitor1']>0.05]['competitor2'].values)
plt.hist(temp,bins=250)
plt.show()

## 3.绩效数据

In [9]:
# 将业绩表现写入data_later
## 1.市场份额比例（每月销量占市场总销量的比例）
def getShare(x):
    return float(x[2]) / dict_month_scale['销量'][x[1]]
## 2.市场份额的绝对值变化
def getMinusShare(x):
    if x[1] == '2015-11': return 0
    lasttime = list_month[list_month.index(x[1])-1]
    lastshare = data_later[(data_later['店铺id']==x[0]) & (data_later['time']==lasttime)]['share'].values
    return x[3]-lastshare[0] if len(lastshare)==1 else 0
## 3.销量的绝对值变化
def getMinusSale(x):
    if x[1] == '2015-11': return 0
    lasttime = list_month[list_month.index(x[1])-1]
    lastsale = data_later[(data_later['店铺id']==x[0]) & (data_later['time']==lasttime)]['销量'].values
    return x[2]-lastsale[0] if len(lastsale)==1 else 0

data_later['share'] = data_later[['店铺id','time','销量']].apply(getShare,axis=1) 
data_later['minus_share'] = data_later[['店铺id','time','销量','share']].apply(getMinusShare,axis=1)
data_later['minus_sale'] = data_later[['店铺id','time','销量']].apply(getMinusSale,axis=1)
data_later = data_later.set_index(['店铺id','time'])

In [10]:
data_later.head()

销量     share  minus_share  minus_sale
店铺id       time                                           
1427469939 2015-11  484  0.018971     0.000000           0
           2015-12  294  0.009149    -0.009822        -190
           2016-01  218  0.005766    -0.003383         -76
           2016-02  112  0.004885    -0.000880        -106
           2016-03  162  0.002625    -0.002261          50

## 4.实证

In [11]:
# 筛选list_shop(面板数据)，考虑自身绩效
list_shop_filter = []
## 以销售量为阈值
sale_limit = 50 ###每月销量阈值
def noSale(shopid):
    ### 后六个月必须每个月都有销量，且每月销量大于n
    data = data_later.loc[shopid, :]
    sales = data[data.index.isin(list_month[-6:])]['销量'].values
    if len(sales) < 6 or min(sales) < sale_limit:
        return True
    return False

## 筛选
for shop in list_shop:
    if noSale(shop): continue
    list_shop_filter.append(shop)
print '筛选后剩余' + str(len(list_shop_filter)) + '家店铺'

筛选后剩余140家店铺


In [20]:
competitor_section = [0.1,0.2,0.3,0.4,1] ##竞争强度区间
## 获得竞争对手绩效之和
def getCompetitorPerformanceSum(shopid, index_section,time, target):
    _sum = 0.0
    data = data_coincide[(data_coincide['competitor1']>=competitor_section[index_section]) \
                         & (data_coincide['competitor1']<competitor_section[index_section+1])]
    competitors = list(data.loc[shopid, :].index)
    for c in competitors:
        try: ##目前这里的报错是因为没有索引值
            _sum += data_later.xs((c, time))[target]
        except:
            _sum += 0
    return _sum
## 获得竞争对手绩效之平均值
def getCompetitorPerformanceAvg(shopid, index_section,time, target):
    _sum = 0.0
    _count = 0
    data = data_coincide[(data_coincide['competitor1']>=competitor_section[index_section]) \
                         & (data_coincide['competitor1']<competitor_section[index_section+1])]
    competitors = list(data.loc[shopid, :].index)
    for c in competitors:
        try: ##目前这里的报错是因为没有索引值
            _sum += data_later.xs((c, time))[target]
            _count += 1
        except:
            _sum += 0
            _count += 0
    if _count == 0: return 0 ##避免除数为0的情况
    return _sum/_count
print 'sum_competitors_performance: ' + str(getCompetitorPerformanceSum(1427469939, 0, '2016-03','minus_sale'))
print 'avg_competitors_performance: ' + str(getCompetitorPerformanceAvg(1427469939, 0, '2016-03','minus_sale'))

sum_competitors_performance: 166.0
avg_competitors_performance: 166.0


In [13]:
## 获得本期绩效
def getNowPerformance(shopid, time, target):
    result = data_later.xs((shopid, time))[target]
    return result
## 获得上期市场占有率
def getLastPerformance(shopid, time, target):
    result = data_later.xs((shopid, list_month[list_month.index(time)-1]))[target]
    return result
print getNowPerformance(1427469939, '2016-03', 'minus_sale')
print getLastPerformance(1427469939, '2016-03', 'minus_sale')

50.0
-106.0


In [14]:
## 获得企业用户规模，做控制变量
def getEnterpriseSale(shopid):
    return sum(data_later.loc[shopid, :]['销量'])
## 获得企业市场规模，做控制变量
def getEnterpriseShare(shopid):
    return sum(data_later.loc[shopid, :]['share'])
print getEnterpriseSale(1427469939)
print getEnterpriseShare(1427469939)

2527
0.0589590522858


In [15]:
## 获得市场规模，做控制变量
def getMarketSaleSum(time):
    return sum(data_later['销量'][:, time])
## 获得平均市场规模，做控制变量
def getMarketSaleAvg(time):
    return np.mean(data_later['销量'][:, time])
def getMarketShareAvg(time):
    return np.mean(data_later['share'][:, time])
print getMarketSaleSum('2016-01')
print getMarketSaleAvg('2016-01')
print getMarketShareAvg('2016-01')

37808
102.183783784
0.0027027027027


In [16]:
_id = 1427469939
time = '2016-01'
target = 'minus_share'
index_section = 0
print 'my_performance: ' + str(getNowPerformance(_id, time, target))
print 'sum_competitors_performance: ' + str(getCompetitorPerformanceSum(_id, index_section, time, target))

my_performance: -0.00338292761026
sum_competitors_performance: -0.0019829408716


In [21]:
# 输出 月份，shopid，自身本期绩效，自身上期绩效，0级竞争者市场份额加总，1级竞争者，2级竞争者..
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\evidence3.csv').decode('utf-8')
f = open(path, 'w')
f.write('time,shopid,performance,enterprise_sale,enterprise_share,'\
        + 'marketsalesum,marketsaleavg,marketshareavg,'\
        + 'competitor0,competitor1,competitor2,competitor3\r\n')
for shopid in list_shop_filter:
    for time in list_month[-6:]:
        print str(time) + ',' + str(shopid) + ',' + str(getNowPerformance(shopid, time,'share'))
        list_competitorsum = [getCompetitorPerformanceAvg(shopid, i, time, 'share') for i in range(4)]
        f.write(time + ',' + str(shopid) + ',' + str(getNowPerformance(shopid, time,'share')/6) + ','\
                + str(getEnterpriseSale(shopid)) + ',' + str(getEnterpriseShare(shopid)) + ','\
                + str(getMarketSaleSum(time)) + ',' + str(getMarketSaleAvg(time)) + ',' + str(getMarketShareAvg(time)) + ','\
                + str(list_competitorsum[0]) + ',' + str(list_competitorsum[1]) + ','\
                + str(list_competitorsum[2]) + ',' + str(list_competitorsum[3]) + '\r\n')
f.close()

2016-05,1459224577,0.00497115422352
2016-06,1459224577,0.00300553786338
2016-07,1459224577,0.00320301569131
2016-08,1459224577,0.00275469984029
2016-09,1459224577,0.00205778528767
2016-10,1459224577,0.00138134438409
2016-05,1530517521,0.00150442825185
2016-06,1530517521,0.00115105705406
2016-07,1530517521,0.000982874851123
2016-08,1530517521,0.00111360206309
2016-09,1530517521,0.00163953624546
2016-10,1530517521,0.00171734707211
2016-05,1581522255,0.0020407896286
2016-06,1581522255,0.00358106639041
2016-07,1581522255,0.00341115389508
2016-08,1581522255,0.00373642797486
2016-09,1581522255,0.00227527478962
2016-10,1581522255,0.00214668384014
2016-05,1583716412,0.00245941314216
2016-06,1583716412,0.00202074460602
2016-07,1583716412,0.00173448503139
2016-08,1583716412,0.00197810892786
2016-09,1583716412,0.00182356582403
2016-10,1583716412,0.00132534393608
2016-05,1551011901,0.00320508627569
2016-06,1551011901,0.00239164076788
2016-07,1551011901,0.00187324383391
2016-08,1551011901,0.0027693

In [22]:
temp = pd.read_csv(path)
# temp[temp['competitor2']>0]
temp

,time,shopid,performance,enterprise_sale,enterprise_share,marketsalesum,marketsaleavg,marketshareavg,competitor0,competitor1,competitor2,competitor3
0,2016-05,1459224577,0.000829,3343,0.070420,76441,126.557947,0.001656,0.000000,0.000000,0.000000,0
1,2016-06,1459224577,0.000501,3343,0.070420,78189,119.555046,0.001529,0.000000,0.000000,0.000000,0
2,2016-07,1459224577,0.000534,3343,0.070420,86481,123.368046,0.001427,0.000000,0.000000,0.000000,0
3,2016-08,1459224577,0.000459,3343,0.070420,68247,92.101215,0.001350,0.000000,0.000000,0.000000,0
4,2016-09,1459224577,0.000343,3343,0.070420,59773,75.566372,0.001264,0.000000,0.000000,0.000000,0
5,2016-10,1459224577,0.000230,3343,0.070420,53571,64.388221,0.001202,0.000000,0.000000,0.000000,0
6,2016-05,1530517521,0.000251,1141,0.021450,76441,126.557947,0.001656,0.009934,0.000000,0.000000,0
7,2016-06,1530517521,0.000192,1141,0.021450,78189,119.555046,0.001529,0.009967,0.000000,0.000000,0
8,2016-07,1530517521,0.000164,1141,0.021450,86481,123.368046,0.001427,0.009147,0.000000,0.000000,0
9,2016-08,1530517521,0.000186,1141,0.021450,68247,92.101215,0.001350,0.006574,0.000000,0.000000,0
